In [ ]:
import  torch
import  torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader
import sys

In [ ]:
#数据预处理
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),#对图像进行随机的crop后再resize成固定的大小
    transforms.RandomRotation(20),#随机选择角度
    transforms.RandomHorizontalFlip(p=0.5),#随机水平翻转
    transforms.ToTensor()
])

In [ ]:
#读取数据
root = 'image'#一个文件夹
train_dataset = datasets.ImageFolder(root+"/train",transform)#train是image的子文件夹
test_dataset = datasets.ImageFolder(root+"/test",transform)

#导入数据
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=8,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=8,shuffle=True)

In [ ]:
a = train_dataset.classes
b = train_dataset.class_to_idx
print(a)
print(b)

In [ ]:
model = models.vgg16(pretrained=True)#使用别人已经训练好的模型
print(model)

In [ ]:
#如果只想训练模型的全连接层，加入以下代码，删除会训练vgg模型的所有参数，包括卷积层
for param in model.parameters():
    param.requires_grad = False
#########################################
#构建新的连接层
model.classifier = torch.nn.Sequential(torch.nn.Linear(25088,100),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(100,2))#25088=224*224,使用交叉熵里面包含了Softmax

In [ ]:
LR = 0.0003 #学习率
#定义代价函数
entrpoy_loss = nn.CrossEntropyLoss()
#定义优化器
optimizer = optim.Adam(model.parameters(),LR)

In [ ]:
def train():
    model.train()
    for i ,data in enumerate(train_loader):
        #获得一个批次的数据和标签
        inputs,lables = data
        #获得模型预测结果
        out = model(inputs) #[64,10]
        #计算loss 使用交叉熵不需要进行独热编码
        loss = entrpoy_loss(out,lables)
        #梯度清零
        optimizer.zero_grad()
        #计算梯度
        loss.backward()
        #修改权值
        optimizer.step()

In [ ]:
def test():
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, lables = data
        # 获得模型的预测结果 [64,10]
        out = model(inputs)
        # 获得最大值以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        correct += (predicted == lables).sum()

    print("Test acc:{0}".format(correct.item() / len(test_dataset)))
    correct = 0
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, lables = data
        # 获得模型的预测结果 [64,10]
        out = model(inputs)
        # 获得最大值以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        correct += (predicted == lables).sum()

    print("Train acc:{0}".format(correct.item() / len(test_dataset)))

In [ ]:
for epoch in range(3):
    print("epoch:",epoch)
    train()
    test()

In [ ]:
torch.save(model.state_dict(),'model/cat_dog.pth')